# python 的 logging 模块日志功能使用详解

**參考來源**

版权声明：本文为CSDN博主「XnCSD」的原创文章，遵循 CC 4.0 BY-SA 版权协议，转载请附上原文出处链接及本声明。

原文链接：https://blog.csdn.net/XnCSD/article/details/87977288

## 、logging 基本用法

### 1、添加日志记录

In [1]:
# 导入 logging 模块
import logging

# 配置 logging 系统
logging.basicConfig(level=logging.DEBUG)

# 添加 log 记录示例
logging.critical('logging critical message.')
logging.error('logging error message')
logging.warning('logging warning message')
logging.info('logging info message')
logging.debug('logging debug message')


CRITICAL:root:logging critical message.
ERROR:root:logging error message
INFO:root:logging info message
DEBUG:root:logging debug message


### 2、日志调用函数的使用

In [26]:
msg = 'foo'
i = 1
logging.info('info message: %s', msg)
logging.info('this is a number: %d', i)

2019-09-22 20:43:40,546 - root - INFO - info message: foo
2019-09-22 20:43:40,563 - root - INFO - this is a number: 1


对于 `error()` 函数，可以传入 `exc_info=True` 参数设置追踪错误信息：

In [27]:
try:
    int('a')
except ValueError:
    logging.error('type error', exc_info=True)

2019-09-22 20:44:32,120 - root - ERROR - type error
Traceback (most recent call last):
  File "<ipython-input-27-6b3e993cb5f9>", line 2, in <module>
    int('a')
ValueError: invalid literal for int() with base 10: 'a'


### 日志输出级别

Level | Numeric value
-|-
CRITICAL | 50
ERROR | 40
WARNING | 30
INFO | 20
DEBUG | 10
NOTSET | 0

`basicConfig()` 的 `level` 参数是一个过滤器，所有等级低于此设定的消息都会被忽略掉，如上面的设置 `level=logging.DEBUG` ，则会输出所有记录。而当输出级别设置为 `ERROR` 时， `level=logging.ERROR` ，只会输出 `critical()` , `error()` 


### 设置日志输出格式

可以通过修改调用 `basicConfig()` 的参数设置日志输出的格式。

* `format` 参数设置输出日志记录的格式，参数的具体格式设置参考 [LogRecord attributes](https://docs.python.org/3/library/logging.html#logrecord-attributes) 

* `datefmt` 设置输出时间的格式，参数格式接受 [time.strftime()](https://docs.python.org/3/library/time.html#time.strftime) 

In [1]:
# --- basicConfig ---
# basicConfig 只能設定一次，記得 restart Kernel


# 导入 logging 模块
import logging

# 配置 logging 系统
logging.basicConfig(
    # filename='example.log', # 指定输出的日志文件
    # stream=sys.stdout, # 系统标准输出 
    level=logging.DEBUG,
    datefmt='%Y/%m/%d %H:%M:%S',
    format='%(asctime)s -- %(name)s -- %(levelname)s -- %(message)s'
)

# 添加 log 记录示例
logging.critical('logging critical message.')
logging.error('logging error message')
logging.warning('logging warning message')
logging.info('logging info message')
logging.debug('logging debug message')


2019/09/22 21:04:02 -- root -- CRITICAL -- logging critical message.
2019/09/22 21:04:02 -- root -- ERROR -- logging error message
2019/09/22 21:04:02 -- root -- WARNING -- logging warning message
2019/09/22 21:04:02 -- root -- INFO -- logging info message
2019/09/22 21:04:02 -- root -- DEBUG -- logging debug message


    filename='example.log', # 指定输出的日志文件
    stream=sys.stdout, # 系统标准输出

注意：`filename` 和 `stream` 不能同时指定，否则会抛出 `ValueError` 异常。若需要设定多种输出方式，可以通过 `handlers` 参数传入一个 `handler` 列表：

```py
from logging import StreamHandler, FileHandler
import sys

sh = StreamHandler(sys.stdout)
fh = FileHandler('example.log')

# 配置 logging 系统
logging.basicConfig(
    handlers=[sh, fh],
    level=logging.DEBUG,
    datefmt='%Y/%m/%d %H:%M:%S',
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
```


## 创建 logger 对象

可以通过 `logging.getLogger(name=None)` 创建一个指定名称的 `Logger` 对象（ `logging` 系统默认的 `logger` 名称为 `root` ）

In [1]:
import logging
import sys

logger = logging.getLogger('testLogger')
logger.setLevel(logging.DEBUG)

# 消息格式化
formatter = logging.Formatter(
    fmt='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y/%m/%d %H:%M:%S'
)

# 日志文件输出
fh = logging.FileHandler('example.log')
fh.setFormatter(formatter)
# 给logger对象添加 handler
logger.addHandler(fh)   

# 系统输出
# StreamHandler
sh = logging.StreamHandler(sys.stdout)
sh.setFormatter(formatter)
logger.addHandler(sh)

logger.critical('logger critical message.')
logger.error('logger error message')
logger.warning('logger warning message')
logger.info('logger info message')
logger.debug('logger debug message')


2019/09/22 21:14:09 - testLogger - CRITICAL - logger critical message.
2019/09/22 21:14:09 - testLogger - ERROR - logger error message
2019/09/22 21:14:09 - testLogger - WARNING - logger warning message
2019/09/22 21:14:09 - testLogger - INFO - logger info message
2019/09/22 21:14:09 - testLogger - DEBUG - logger debug message


## 三、给库或者模块添加日志记录

对于想执行日志记录的库来说，应该创建一个专用的日志对象并将其初始化为 `NullHandler`

**不对已有的日志配置做任何假设**

In [2]:
%%file testlib.py

import logging

logger = logging.getLogger(__name__)
logger.addHandler(logging.NullHandler())


def foo():
    logger.info('info: this is foo message')
    print('print: foo')


Writing testlib.py


`getLogger(__name__)` 创建一个与模块同名的一个专有的日志对象，由于所有模块是唯一的，这样就于其他日志对象隔开了

`logger.addHandler(logging.NullHandler())` 操作绑定一个空的处理例程到刚才的日志对象。默认情况下，空 `handler` 会忽略所有日志消息。因此，用到这个库且日志系统从未配置过，那么就不会出现任何日志信息或警告信息。

In [3]:
import testlib
testlib.foo()

print: foo


但是如果日志系统得到适当的配置，则日志消息将开始出现

In [4]:
import logging
import testlib
logging.basicConfig(level=logging.DEBUG)

testlib.foo()

INFO:testlib:info: this is foo message


print: foo


在调用模块的程序中也能定义自身的 `logger`

In [1]:

import logging
import testlib
logging.basicConfig(level=logging.DEBUG)

logger = logging.getLogger('testLogger')

testlib.foo()
logger.info('this is test logger info')


INFO:testlib:info: this is foo message
INFO:testLogger:this is test logger info


print: foo


## 使用配置文件设置日志的配置信息

上面的日志配置都被直接硬编码到了程序中，这么做不是好的编程规范，`logging.config` ，提供了 `logging.config.fileConfig()` 方法可以从配置文件中进行日志配置

创建一个配置文件 `logconfig.ini`

In [2]:
%%file logconfig.ini
[loggers]
keys=root

[handlers]
keys=consoleHandler

[formatters]
keys=simpleFormatter

[logger_root]
level=DEBUG
handlers=consoleHandler

[handler_consoleHandler]
class=StreamHandler
level=DEBUG
formatter=simpleFormatter
args=(sys.stdout,)

[formatter_simpleFormatter]
format=%(name)s - %(levelname)s - %(message)s


Writing logconfig.ini


`[loggers]` 配置项指定需要配置的 `logger` ， `root` 是 `logging` 系统默认`logger` 。若有多个值，用逗号分隔 `keys=root` , `testLogger` 。然后在`[logger_%(name)s]` 配置项配置各个 `logger` 的具体配置信息

請參考 `logger` 創建

In [3]:
import logging
import logging.config


logging.config.fileConfig('logconfig.ini')

logging.critical('logger critical message.')
logging.error('logger error message')
logging.warning('logger warning message')
logging.info('logger info message')
logging.debug('logger debug message')


root - CRITICAL - logger critical message.
root - ERROR - logger error message
root - WARNING - logger warning message
root - INFO - logger info message
root - DEBUG - logger debug message


输出设置为日志文件，且错误信息单独输出到错误日志中

```ini
[loggers]
keys=root

[handlers]
keys=fileHandler, errorFileHandler

[formatters]
keys=simpleFormatter

[logger_root]
level=DEBUG
handlers=fileHandler, errorFileHandler

[handler_fileHandler]
class=FileHandler
level=DEBUG
formatter=simpleFormatter
args=('example.log', 'a')

[handler_errorFileHandler]
class=FileHandler
level=ERROR
formatter=simpleFormatter
args=('example-error.log', 'a')

[formatter_simpleFormatter]
format=%(name)s - %(levelname)s - %(message)s
```


# 多模块使用logging

**參考來源**

版权声明：本文为CSDN博主「chosen0ne」的原创文章，遵循 CC 4.0 BY-SA 版权协议，转载请附上原文出处链接及本声明。

原文链接：https://blog.csdn.net/chosen0ne/article/details/7319306


## logging 使用

In [4]:

import logging
import logging.handlers
 
LOG_FILE = 'tst.log'

# 实例化handler 
handler = logging.handlers.RotatingFileHandler(
    LOG_FILE,
    maxBytes = 1024*1024,
    backupCount = 5
) 

fmt = '%(asctime)s - %(filename)s:%(lineno)s - %(name)s - %(message)s'

# 实例化 formatter
formatter = logging.Formatter(fmt)    
# 为 handler 添加 formatter
handler.setFormatter(formatter)     

# 获取名为tst的logger
logger = logging.getLogger('tst')   
# 为 logger 添加 handler
logger.addHandler(handler)   

logger.setLevel(logging.DEBUG)
 
logger.info('first info message')
logger.debug('first debug message')

INFO:tst:first info message
DEBUG:tst:first debug message


In [ ]:
# %load tst.log
2019-09-21 21:09:05,216 - <ipython-input-4-a7a793a09af4>:17 - tst - first info message
2019-09-21 21:09:05,218 - <ipython-input-4-a7a793a09af4>:18 - tst - first debug message


## 多模块使用logging


**配置文件**

In [12]:
%%file  loggin.conf
[loggers]
keys=root,main
 
[handlers]
keys=consoleHandler,fileHandler
 
[formatters]
keys=fmt
 
[logger_root]
level=DEBUG
handlers=consoleHandler
 
[logger_main]
level=DEBUG
qualname=main
handlers=fileHandler
 
[handler_consoleHandler]
class=StreamHandler
level=DEBUG
formatter=fmt
args=(sys.stdout,)
 
[handler_fileHandler]
class=logging.handlers.RotatingFileHandler
level=DEBUG
formatter=fmt
args=('tst.log','a',20000,5,)
 
[formatter_fmt]
format=%(asctime)s - %(name)s - %(levelname)s - %(message)s
datefmt=

Overwriting loggin.conf


In [19]:
%%file main.py
import logging
import logging.config

from os import path

log_file_path = path.join(
    path.dirname(path.abspath(__file__)),
    'loggin.conf'
)

logging.config.fileConfig(log_file_path)
# 有時找不到 logging.conf，以上述方法指定絕對位置
#logging.config.fileConfig('logging.conf')

root_logger = logging.getLogger('root')
root_logger.debug('test root logger...')
 
logger = logging.getLogger('main')
logger.info('test main logger')

logger.info('start import module \'mod\'...')
import mod
 
logger.debug('let\'s test mod.testLogger()')
mod.testLogger()
root_logger.info('finish test...')


Overwriting main.py


In [14]:
%%file mod.py
import logging
import submod
 
logger = logging.getLogger('main.mod')
logger.info('logger of mod say something...')
 
def testLogger():
    logger.debug('this is mod.testLogger...')
    submod.tst()


Writing mod.py


In [21]:
%%file submod.py
import logging
 
logger = logging.getLogger('main.mod.submod')
logger.info('logger of submod say something...')
 
def tst():
    logger.info('this is submod.tst()...')


Writing submod.py


In [22]:
%run main.py

2019-09-21 21:22:22,887 - main - INFO - test main logger
2019-09-21 21:22:22,888 - main - INFO - start import module 'mod'...
2019-09-21 21:22:22,898 - main.mod.submod - INFO - logger of submod say something...
2019-09-21 21:22:22,899 - main.mod - INFO - logger of mod say something...
2019-09-21 21:22:22,900 - main - DEBUG - let's test mod.testLogger()
2019-09-21 21:22:22,901 - main.mod - DEBUG - this is mod.testLogger...
2019-09-21 21:22:22,902 - main.mod.submod - INFO - this is submod.tst()...


In [ ]:
# %load tst.log
2019-09-21 21:09:05,216 - <ipython-input-4-a7a793a09af4>:17 - tst - first info message
2019-09-21 21:09:05,218 - <ipython-input-4-a7a793a09af4>:18 - tst - first debug message
2019-09-21 21:22:08,853 - main - INFO - test main logger
2019-09-21 21:22:08,853 - main - INFO - start import module 'mod'...
2019-09-21 21:22:22,887 - main - INFO - test main logger
2019-09-21 21:22:22,888 - main - INFO - start import module 'mod'...
2019-09-21 21:22:22,898 - main.mod.submod - INFO - logger of submod say something...
2019-09-21 21:22:22,899 - main.mod - INFO - logger of mod say something...
2019-09-21 21:22:22,900 - main - DEBUG - let's test mod.testLogger()
2019-09-21 21:22:22,901 - main.mod - DEBUG - this is mod.testLogger...
2019-09-21 21:22:22,902 - main.mod.submod - INFO - this is submod.tst()...


`tst.log` 中没有 `root logger` 输出的信息，因为 `logging.conf` 中配置了只有`main logger` 及其子 `logger` 使用 `RotatingFileHandler`

# 補充(未看)

## logging遇到多进程（important）

https://blog.csdn.net/qq_27437781/article/details/83417841